### **Python packages**

In [36]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Workspace

### **Workspace**

In [37]:
ws = Workspace.from_config(path="./Config/config.json")

### **Train a simple model**

I dati vanno inseriti nella stessa cartella nella quale si trova lo script.

In [45]:
#--- Define a conda env.
sklearn_env = Environment("sklearn-env")

# posso aggiungere dei pacchetti se non sono presenti.
packages = CondaDependencies.create(conda_packages=["scikit-learn", "pip", "pandas"],
                                    pip_packages=["azureml-defaults"]
                                    )
sklearn_env.python.conda_dependencies = packages

# Create a script config.
script_config = ScriptRunConfig(source_directory="./src",
                                script="2_Tutorial.py",
                                environment=sklearn_env)

# Submit the experiment
exp = Experiment(workspace = ws, name = "exp-training")
run = exp.submit(config = script_config)

run.wait_for_completion()

{'runId': 'exp-training_1652108507_3a81b1d5',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-05-09T15:01:49.578002Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'ffa3c925-958a-4d61-a78a-ff176512010c',
  'azureml.git.repository_uri': 'https://github.com/LuciaRavazzi/AzureML',
  'mlflow.source.git.repoURL': 'https://github.com/LuciaRavazzi/AzureML',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '501dd29e398df4697b4a4938b90b91bfb1fdb1f9',
  'mlflow.source.git.commit': '501dd29e398df4697b4a4938b90b91bfb1fdb1f9',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': '2_Tutorial.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacache

### **Train model with some arguments**

Standard practice is for scripts that train models to save them in the run's outputs folder.

In [62]:
# Create a script config.

for reg_rate in [0.0001, 0.01, 0.1, 0.5, 0.9]:
    script_config = ScriptRunConfig(source_directory="./src",
                                    script="2_Train_args.py",
                                    arguments = ['--reg-rate', reg_rate],
                                    environment=sklearn_env)

    # Submit the experiment
    exp = Experiment(workspace = ws, name = "exp-training")
    run = exp.submit(config = script_config)

    run.wait_for_completion()

In [65]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/70_driver_log.txt',
 'logs/azureml/7820_azureml.log',
 'outputs/model.pkl']

In [64]:
run.get_metrics()

{'Accuracy': 0.7619047619047619, 'reg': 0.9}

### **Retrieve models**

In this way, you can retrieve a model and make inference.


In [53]:
# files generated by the last run.
for file in run.get_file_names():
    print(file)

# Download a named file
run.download_file(name='outputs/model.pkl', output_file_path='model_train.pkl')

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/3644_azureml.log
outputs/model.pkl


### **Register models**

In [54]:
from azureml.core import Model

In [58]:
model = Model.register(workspace=ws,
                       model_name = 'classification_model',
                       model_path='src/outputs/model.pkl',
                       description="Register a trained model",
                       tags = {'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model classification_model


In [59]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)

classification_model version: 5
microcard_production_model version: 2
microcard_production_model version: 1
bwood_production_model version: 3
bwood_production_model version: 2
bwood_dev_model version: 1
bwood_production_model version: 1
my_test_model version: 7
dev_production_model version: 1
production_model version: 1
my_test_model version: 6
my_test_model version: 5
my_test_model version: 4
my_test_model version: 3
ElsticNet version: 2
ElsticNet version: 1
my_dev_model version: 1
my_production_model version: 1
my_test_model version: 2
umap-model-v2 version: 2
test-2 version: 1
test version: 2
test version: 1
umap-model-v1 version: 3
test_new_ocr_exp_finale_pickle-v1 version: 1
umap-model-v1 version: 2
dotted_fonts version: 1
industrial_fonts version: 1
embedding_industrial_pickle version: 1
embedding_industrial version: 1
embedding_dotted_pickle version: 1
embedding_dotted version: 1
synthetic_dataset version: 1
student_model version: 1
craft_model version: 1
dino_vitb_8 version: 1
